# H1 DATA 512 -- Homework 2
### Keegan Freeman
### 10/11/22

# *Data Merging*
**import libraries and read csv**

In [1]:
# imports
import json, time, urllib.parse
import requests
import pandas as pd
import numpy as np

poly = pd.read_csv("politicians_international_SEPT.2022.csv")
pop = pd.read_csv("population_by_country_2022.csv")

**Create Dataframe combining politition along with associated country and population**

**Take care of 'Korean' outlier present in polititions_by_country.csv by renaming to 'Korea' and defining population as the addition of North and South Korea's populations**


In [2]:
poly_pop = pd.DataFrame(poly["name"])
poly_pop["country"] = poly["country"]
poly_pop["region"] = poly["country"]

### Outlier -- Korean
poly_pop["region"].replace("Korean", "Korea", inplace=True)
poly_pop["country"].replace("Korean", "Korea", inplace=True)
NK = float(pop[pop["Geography"] == "Korea, North"]["Population (millions)"])
SK = float(pop[pop["Geography"] == "Korea, South"]["Population (millions)"])
pop = pop.append({"Geography":"EAST ASIA","Population (millions)": 1674}, ignore_index=True)
pop = pop.append({"Geography":"Korea","Population (millions)": float(NK+SK)}, ignore_index=True)
###

poly_pop = poly_pop.merge(pop, how='left', left_on="country", right_on="Geography")
poly_pop["region"] = poly["country"]

**Associate each politition with their associated region using last region memory within for loop**

**Take care of our 'Korea' outlier by associating region with same as North Korea's**

**Drop 'Geography' from df as it's redundant**

**Create dummy variables for revision_id and article quality in the df**

In [3]:
dic = {}
is_super = False
curr_region = ""
for i in pop["Geography"]:
    if(i.isupper()):
        curr_region = i
    else:
        dic[i] = curr_region

### Outlier -- Korea
dic["Korea"] = pop[pop["Geography"] == "Korea, North"]["Geography"]
###        

poly_pop = poly_pop.replace({"region": dic})
poly_pop.drop(labels="Geography", axis=1, inplace=True)

poly_pop["revision_id"] = [-1]*len(poly_pop)
poly_pop["article_quality"] = ["Unknown"]*len(poly_pop)

print(poly_pop)

                       name      country          region  \
0           Shahjahan Noori  Afghanistan      SOUTH ASIA   
1     Abdul Ghafar Lakanwal  Afghanistan      SOUTH ASIA   
2            Majah Ha Adrif  Afghanistan      SOUTH ASIA   
3         Haroon al-Afghani  Afghanistan      SOUTH ASIA   
4               Tayyab Agha  Afghanistan      SOUTH ASIA   
...                     ...          ...             ...   
7579        Rekayi Tangwena     Zimbabwe  EASTERN AFRICA   
7580       Josiah Tongogara     Zimbabwe  EASTERN AFRICA   
7581      Langton Towungana     Zimbabwe  EASTERN AFRICA   
7582    Herbert Ushewokunze     Zimbabwe  EASTERN AFRICA   
7583           Denis Walker     Zimbabwe  EASTERN AFRICA   

      Population (millions)  revision_id article_quality  
0                      41.1           -1         Unknown  
1                      41.1           -1         Unknown  
2                      41.1           -1         Unknown  
3                      41.1           -1   

# Obtaining Article Quality Predictions

## Finding Last Revision ID

**Create constants needed for webscraping API**

*Note: 'User-Agent' should be altered for replicaiton or reproduction to author's creditials as should 'ARTICLE_TITLES' for replication*

In [4]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'kafreem3@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2022',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = poly["name"]

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

Create resusable function such that API request can be made easily and with different parameterized attributes. This heavily relies on constants defined in cell above. If above cell above is properly altered for replication then only article_title should be altered as well.

In [5]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    request_template['titles'] = article_title
        
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


**Pull data from Wikapedia for each politician and take *only* 'lastrevid' for finding article quality later on**

We first create a list for 'rev_id' and fill it with information taken. Once web scraping is done, we can create our 'revision_id' column for the df and associate it with the list.

*Note: I was curious and added a function to record the time taken to complete web-scrape.*

*Time Warning: This will take ~30 minutes to complete*

In [6]:
print("Obtaining Article Revision IDs...")

start = time.time()

rev_ids = []

for i in poly_pop["name"]:
    
    try:
        info = request_pageinfo_per_article(i)
        key = list(info["query"]["pages"].keys())[0]
        rev_id = info["query"]["pages"][key]["lastrevid"]
    except(IndexError, KeyError) as error:
        rev_id = -1
    
    #poly_pop.loc[(poly_pop["name"] == i).index[0],"revision_id"] = rev_id
    rev_ids.append(rev_id)
    
end = time.time()
        
print("...Process Completed")
print("Webscrape completed in " + str((end-start)/60) + " minutes.")
poly_pop["revision_id"] = rev_ids

Obtaining Article Revision IDs...
...Process Completed
Webscrape completed in 32.83729234933853 minutes.


**Print to verify that revision_ids look correct**

In [7]:
print(poly_pop)

                       name      country          region  \
0           Shahjahan Noori  Afghanistan      SOUTH ASIA   
1     Abdul Ghafar Lakanwal  Afghanistan      SOUTH ASIA   
2            Majah Ha Adrif  Afghanistan      SOUTH ASIA   
3         Haroon al-Afghani  Afghanistan      SOUTH ASIA   
4               Tayyab Agha  Afghanistan      SOUTH ASIA   
...                     ...          ...             ...   
7579        Rekayi Tangwena     Zimbabwe  EASTERN AFRICA   
7580       Josiah Tongogara     Zimbabwe  EASTERN AFRICA   
7581      Langton Towungana     Zimbabwe  EASTERN AFRICA   
7582    Herbert Ushewokunze     Zimbabwe  EASTERN AFRICA   
7583           Denis Walker     Zimbabwe  EASTERN AFRICA   

      Population (millions)  revision_id article_quality  
0                      41.1   1099689043         Unknown  
1                      41.1    943562276         Unknown  
2                      41.1    852404094         Unknown  
3                      41.1   1095102390   

**Save as benchmark such that you don't have to re-run first webscrape in case of notebook error**

In [8]:
poly_pop.to_csv('benchmark.csv')

## Finding Article Quality Prediction from Last Revision ID


**Create constants needed for webscraping API**

*Note: 'User-Agent' should be altered for replicaiton or reproduction to author's creditials as should 'ARTICLE_TITLES' for replication*

In [9]:
#########
#
#    CONSTANTS
#

# The current ORES API endpoint
API_ORES_SCORE_ENDPOINT = "https://ores.wikimedia.org/v3"
# A template for mapping to the URL
API_ORES_SCORE_PARAMS = "/scores/{context}/{revid}/{model}"

# Use some delays so that we do not hammer the API with our requests
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'kafreem3@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2022'
}

# A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
ARTICLE_REVISIONS = poly["name"]

# This template lists the basic parameters for making an ORES request
ORES_PARAMS_TEMPLATE = {
    "context": "enwiki",        # which WMF project for the specified revid
    "revid" : "",               # the revision to be scored - this will probably change each call
    "model": "articlequality"   # the AI/ML scoring model to apply to the reviewion
}
#
# The current ML models for English wikipedia are:
#   "articlequality"
#   "articletopic"
#   "damaging"
#   "version"
#   "draftquality"
#   "drafttopic"
#   "goodfaith"
#   "wp10"
#
# The specific documentation on these is scattered so if you want to use one you'll have to look around.
#

Create resusable function such that API request can be made easily and with different parameterized attributes. This heavily relies on constants defined in cell above. If above cell above is properly altered for replication then only article_title should be altered as well.

In [10]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, 
                                   endpoint_url = API_ORES_SCORE_ENDPOINT, 
                                   endpoint_params = API_ORES_SCORE_PARAMS, 
                                   request_template = ORES_PARAMS_TEMPLATE,
                                   headers = REQUEST_HEADERS,
                                   features=False):
    # Make sure we have an article revision id
    if not article_revid: return None
    
    # set the revision id into the template
    request_template['revid'] = article_revid
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # the features used by the ML model can sometimes be returned as well as scores
    if features:
        request_url = request_url+"?features=true"
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


**Pull data from Wikapedia for each politician and take *only* 'article_quality for later analysis**

We first create a list for 'article_score' and fill it with information taken. Once web scraping is done, we can create our 'article_quality' column for the df and associate it with the list.

*Note: I was curious and added a function to record the time taken to complete web-scrape.*

*Time Warning: This will take ~25 minutes to complete*

In [11]:
print("Obtaining Article Quality Predictions...")

scores = []
start = time.time()

for i in rev_ids:
    
    try:
        info = request_ores_score_per_article(i)
        pred = info["enwiki"]["scores"][str(i)]["articlequality"]["score"]["prediction"]
    except(IndexError, KeyError, TypeError) as error:
        pred = "Unknown"
    
    scores.append(pred)
    
end = time.time()
        
print("...Process completed")
print("Webscrape completed in " + str((end-start)/60) + " minutes.")

poly_pop["article_quality"] = scores

Obtaining Article Quality Predictions...
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
...Process completed
Webscrape completed in 28.304612525304158 minutes.


**Print our *complete* dataframe to look for outstanding issues**

In [12]:
print(poly_pop)

                       name      country          region  \
0           Shahjahan Noori  Afghanistan      SOUTH ASIA   
1     Abdul Ghafar Lakanwal  Afghanistan      SOUTH ASIA   
2            Majah Ha Adrif  Afghanistan      SOUTH ASIA   
3         Haroon al-Afghani  Afghanistan      SOUTH ASIA   
4               Tayyab Agha  Afghanistan      SOUTH ASIA   
...                     ...          ...             ...   
7579        Rekayi Tangwena     Zimbabwe  EASTERN AFRICA   
7580       Josiah Tongogara     Zimbabwe  EASTERN AFRICA   
7581      Langton Towungana     Zimbabwe  EASTERN AFRICA   
7582    Herbert Ushewokunze     Zimbabwe  EASTERN AFRICA   
7583           Denis Walker     Zimbabwe  EASTERN AFRICA   

      Population (millions)  revision_id article_quality  
0                      41.1   1099689043              GA  
1                      41.1    943562276           Start  
2                      41.1    852404094           Start  
3                      41.1   1095102390   

**Print the counts for each article rating for outstanding issues**

In [13]:
print(poly_pop['article_quality'].value_counts())

Stub       3099
Start      2832
C          1283
GA          242
B            75
FA           45
Unknown       8
Name: article_quality, dtype: int64


**Again, save as a benchmark so we don't have to re-run web-scraping in case of error**

In [14]:
poly_pop.to_csv('benchmark.csv')

**Find articles that don't have an associated score. Print these out of curiosity**

In [15]:
outliers_idx = np.where(poly_pop['article_quality'] == "Unknown")[0]
outliers = poly_pop.take(outliers_idx)
print(outliers)

                          name       country           region  \
2446        Prince Ofosu Sefah         Ghana   WESTERN AFRICA   
2985      Harjit Kaur Talwandi         India       SOUTH ASIA   
3212   Abd al-Razzaq al-Hasani          Iraq     WESTERN ASIA   
3784              Kang Sun-nam  Korea, North        EAST ASIA   
4865  Abiodun Abimbola Orekoya       Nigeria   WESTERN AFRICA   
4879  Segun “Aeroland” Adewale       Nigeria   WESTERN AFRICA   
5801            Roman Konoplev        Russia   EASTERN EUROPE   
6344    Nhlanhla “Lux” Dlamini  South Africa  SOUTHERN AFRICA   

      Population (millions)  revision_id article_quality  
2446                   33.5           -1         Unknown  
2985                 1417.2           -1         Unknown  
3212                   44.5           -1         Unknown  
3784                   26.1           -1         Unknown  
4865                  218.5           -1         Unknown  
4879                  218.5           -1         Unknown  
5

**Remove outliers from our dataframe, and save 'cleaned' df as .csv**

In [16]:
poly_pop = poly_pop.drop(outliers_idx, axis=0)
print(poly_pop)
poly_pop.to_csv('wp_politicians_by_country.csv')

                       name      country          region  \
0           Shahjahan Noori  Afghanistan      SOUTH ASIA   
1     Abdul Ghafar Lakanwal  Afghanistan      SOUTH ASIA   
2            Majah Ha Adrif  Afghanistan      SOUTH ASIA   
3         Haroon al-Afghani  Afghanistan      SOUTH ASIA   
4               Tayyab Agha  Afghanistan      SOUTH ASIA   
...                     ...          ...             ...   
7579        Rekayi Tangwena     Zimbabwe  EASTERN AFRICA   
7580       Josiah Tongogara     Zimbabwe  EASTERN AFRICA   
7581      Langton Towungana     Zimbabwe  EASTERN AFRICA   
7582    Herbert Ushewokunze     Zimbabwe  EASTERN AFRICA   
7583           Denis Walker     Zimbabwe  EASTERN AFRICA   

      Population (millions)  revision_id article_quality  
0                      41.1   1099689043              GA  
1                      41.1    943562276           Start  
2                      41.1    852404094           Start  
3                      41.1   1095102390   

# Analysis

**Group df by both country and region. For each region or country, fill dictionaries with associated article quantity per capita (divide by population) and 'quality' article quantity per capita (divide by population)**

*Note: 'Quality' article is defined by being either a 'featured article' or 'good article'*

In [17]:
grouped_country = poly_pop.groupby("country")
grouped_region = poly_pop.groupby("region")

con_dic = {}
reg_dic = {}

for geo in pop["Geography"]:
    try:
        arr = grouped_country.get_group(geo)
        good_art_len = len(arr[(arr['article_quality']=="FA") | (arr['article_quality']=="GA")])
        
        per_cap_cov = len(arr)/(arr["Population (millions)"].iloc[0]*1000000)
        per_cap_qua = good_art_len/(arr["Population (millions)"].iloc[0]*1000000)
        
        lst = [per_cap_cov, per_cap_qua]
        con_dic[geo] = lst
        
    except KeyError:
        try:
            arr = grouped_region.get_group(geo)
            good_art_len = len(arr[(arr['article_quality']=="FA") | (arr['article_quality']=="GA")])
            
            per_cap_cov = len(arr)/(arr["Population (millions)"].iloc[0]*1000000)
            per_cap_qua = good_art_len/(arr["Population (millions)"].iloc[0]*1000000)
            
            lst = [per_cap_cov, per_cap_qua]
            reg_dic[geo] = lst
            
        except KeyError:
            continue

C:\Users\kafre\AppData\Local\Temp/ipykernel_10948/3251348182.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  per_cap_cov = len(arr)/(arr["Population (millions)"].iloc[0]*1000000)
C:\Users\kafre\AppData\Local\Temp/ipykernel_10948/3251348182.py:13: RuntimeWarning: invalid value encountered in double_scalars
  per_cap_qua = good_art_len/(arr["Population (millions)"].iloc[0]*1000000)
C:\Users\kafre\AppData\Local\Temp/ipykernel_10948/3251348182.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  per_cap_qua = good_art_len/(arr["Population (millions)"].iloc[0]*1000000)


**Find any values (quality per capita and quantity per capita) within each dictionary that are either NaN or inf**

For each value found, print associated country and store them in wp_countries-no_match.txt as stipulated within Homework 2 -- **"Identify all countries for which there are no matches and output a list of those countries, with each country on a separate line"**

Finally, remove these found values associated key-value pair from their associate dictionary

In [18]:
import math

con_keys_to_kill = []
reg_keys_to_kill = []

for key in con_dic.keys():
        if(math.isinf(con_dic[key][0]) or math.isinf(con_dic[key][1]) or math.isnan(con_dic[key][0]) or math.isnan(con_dic[key][1])):
            con_keys_to_kill.append(key)
            
for key in reg_dic.keys():
        if(math.isinf(reg_dic[key][0]) or math.isinf(reg_dic[key][1]) or math.isnan(reg_dic[key][0]) or math.isnan(reg_dic[key][1])):
            reg_keys_to_kill.append(key)

with open('wp_countries-no_match.txt', 'w', encoding='utf-8') as f:
    for key in con_keys_to_kill:
        print(key)
        f.write(key)
        f.write('\n')
        con_dic.pop(key)
    
for key in reg_keys_to_kill:
    reg_dic.pop(key)

Liechtenstein
Monaco
San Marino
Nauru
Palau
Tuvalu


**Sort each dictionary by both quantity per capita as well as quality per capita**

In [19]:
sorted_con_coverage = sorted(con_dic.items(), key=lambda x: x[1])
sorted_reg_coverage = sorted(reg_dic.items(), key=lambda x: x[1])

sorted_con_quality = sorted(con_dic.items(), key=lambda x: x[1][1])
sorted_reg_quality = sorted(reg_dic.items(), key=lambda x: x[1][1])

**Remove countries that have 0 quality articles**

In [20]:
temp = []
i = 0
for el in sorted_con_quality:
    if(el[1][1] == 0):
        #sorted_con_quality.pop(el)
        sorted_con_quality.pop(i)
    else:
        temp.append(el)
    i+= 1
sorted_con_quality = temp

# Results

## Top 10 countries by coverage

In [21]:
top_count_by_coverage = pd.DataFrame(columns=["Country", "Articles per Capita"])

for i in range(10):
    i = i+1
    el = sorted_con_coverage[-i]
    row = {"Country": el[0], "Articles per Capita": el[1][0]}
    top_count_by_coverage = top_count_by_coverage.append(row, ignore_index=True)

display(top_count_by_coverage)

,Country,Articles per Capita
0,Antigua and Barbuda,0.000170
1,Federated States of Micronesia,0.000130
2,Andorra,0.000100
3,Barbados,0.000093
4,Marshall Islands,0.000090
5,Montenegro,0.000060
6,Seychelles,0.000060
7,Luxembourg,0.000053
8,Bhutan,0.000051
9,Grenada,0.000050


## Bottom 10 countries by coverage

In [22]:
bottom_count_by_coverage = pd.DataFrame(columns=["Country", "Articles per Capita"])

for i in range(10):
    el = sorted_con_coverage[i]
    row = {"Country": el[0], "Articles per Capita": el[1][0]}
    bottom_count_by_coverage = bottom_count_by_coverage.append(row, ignore_index=True)

display(bottom_count_by_coverage)

,Country,Articles per Capita
0,China,1.392176e-09
1,Mexico,7.843137e-09
2,Saudi Arabia,8.174387e-08
3,Romania,1.052632e-07
4,India,1.255998e-07
5,Sri Lanka,1.339286e-07
6,Egypt,1.352657e-07
7,Ethiopia,2.025932e-07
8,Taiwan,2.155172e-07
9,Vietnam,2.716298e-07


## Top 10 countries by quality per capita

In [23]:
top_count_by_quality = pd.DataFrame(columns=["Country", "Quality Articles per Capita"])

for i in range(10):
    i = i+1
    el = sorted_con_quality[-i]
    row = {"Country": el[0], "Quality Articles per Capita": el[1][1]}
    top_count_by_quality = top_count_by_quality.append(row, ignore_index=True)

display(top_count_by_quality)

,Country,Quality Articles per Capita
0,Andorra,2.000000e-05
1,Montenegro,5.000000e-06
2,Albania,2.142857e-06
3,Suriname,1.666667e-06
4,Bosnia-Herzegovina,1.470588e-06
5,Lithuania,1.071429e-06
6,Croatia,1.052632e-06
7,Slovenia,9.523810e-07
8,Palestinian Territory,9.259259e-07
9,Gabon,8.333333e-07


## Bottom 10 countries by quality per capita

In [24]:
bottom_count_by_quality = pd.DataFrame(columns=["Country", "Quality Articles per Capita"])

for i in range(10):
    el = sorted_con_quality[i]
    row = {"Country": el[0], "Quality Articles per Capita": el[1][1]}
    bottom_count_by_quality = bottom_count_by_quality.append(row, ignore_index=True)

display(bottom_count_by_quality)

,Country,Quality Articles per Capita
0,India,4.233700e-09
1,Thailand,1.497006e-08
2,Japan,1.601281e-08
3,Nigeria,1.830664e-08
4,Vietnam,2.012072e-08
5,Colombia,2.036660e-08
6,Uganda,2.118644e-08
7,Pakistan,2.120441e-08
8,Sudan,2.132196e-08
9,Iran,2.257336e-08


## Graphic regions by total coverage

In [25]:
top_reg_by_coverage = pd.DataFrame(columns=["Country", "Articles per Capita"])

for i in range(len(sorted_reg_coverage)):
    i+= 1
    el = sorted_reg_coverage[-i]
    row = {"Country": el[0], "Articles per Capita": el[1][0]}
    top_reg_by_coverage = top_reg_by_coverage.append(row, ignore_index=True)

display(top_reg_by_coverage)

,Country,Articles per Capita
0,CARIBBEAN,2.010000e-03
1,OCEANIA,8.600000e-04
2,CENTRAL AMERICA,4.875000e-04
3,SOUTHERN EUROPE,3.178571e-04
4,WESTERN ASIA,2.286667e-04
5,EASTERN EUROPE,7.989130e-05
6,WESTERN EUROPE,7.766667e-05
7,EASTERN AFRICA,5.038760e-05
8,SOUTHERN AFRICA,4.500000e-05
9,NORTHERN EUROPE,4.440678e-05


## Graphic regions by high quality coverage

In [26]:
top_reg_by_quality = pd.DataFrame(columns=["Country", "Articles per Capita"])

for i in range(len(sorted_reg_quality)):
    i+=1
    el = sorted_reg_quality[-i]
    row = {"Country": el[0], "Articles per Capita": el[1][1]}
    top_reg_by_quality = top_reg_by_quality.append(row, ignore_index=True)

display(top_reg_by_quality)

,Country,Articles per Capita
0,CARIBBEAN,8.000000e-05
1,CENTRAL AMERICA,2.500000e-05
2,OCEANIA,2.000000e-05
3,SOUTHERN EUROPE,1.642857e-05
4,WESTERN ASIA,9.333333e-06
5,EASTERN EUROPE,4.130435e-06
6,WESTERN EUROPE,2.444444e-06
7,SOUTHERN AFRICA,1.538462e-06
8,SOUTHEAST ASIA,1.428571e-06
9,NORTHERN EUROPE,1.355932e-06
